In [ ]:
import cv2
import imutils
import pytesseract
from matplotlib import pyplot as plt

In [ ]:

# Tesseract Binary Installation path
pytesseract.pytesseract.tesseract_cmd = "D:\\Program Files\\Tesseract-OCR\\tesseract"

In [ ]:
#Load the image
img = cv2.imread("plate.jpg") # Source: https://www.autobild.es/practicos/como-saber-letra-va-matricula-coches-581133

# Resize the image so it is faster to work with it
img = imutils.resize(img, width=300)

In [ ]:
# Convert it to grayscale
grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# Remove noise by applying Bilateral Filter. More info: https://dsp.stackexchange.com/questions/8316/the-difference-between-bilateral-filter-and-gaussian-filter
smooth_img = cv2.bilateralFilter(grayscale_img, 11, 11, 17)

In [ ]:
# Find the edges
edges_only = cv2.Canny(img, 20, 300)
plt.imshow(cv2.cvtColor(edges_only, cv2.COLOR_BGR2RGB))

In [ ]:
import numpy as np
kernel = np.ones((2,2),np.uint8)
edges_only = cv2.dilate(edges_only,kernel,iterations = 1)

plt.imshow(cv2.cvtColor(edges_only, cv2.COLOR_BGR2RGB))

In [ ]:
#Find countours in image
cnts,new = cv2.findContours(edges_only, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
#Display the countours
img_with_contours=img.copy()
cv2.drawContours(img_with_contours,cnts,-1,(0,255,0),1)
plt.imshow(cv2.cvtColor(img_with_contours, cv2.COLOR_BGR2RGB))

In [ ]:
# Filter countours
sorted_cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:20]
screenCnt = None


In [ ]:
# Display filtered contours
filtered_img = img.copy()
cv2.drawContours(filtered_img,sorted_cnts,-1,(0,200,0),1)
plt.imshow(cv2.cvtColor(filtered_img, cv2.COLOR_BGR2RGB))

In [ ]:
i=7
final_img = None
for c in sorted_cnts:
    perimeter = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018* perimeter, True)
    area = cv2.contourArea(c)
    if len(approx) == 4 and area>100: 
        screenCnt = approx
        x,y,w,h = cv2.boundingRect(c)
        final_img=img[y:y+h,x:x+w]
        i+=1
        break
plt.imshow(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB))

In [ ]:

#Read from image
plate = pytesseract.image_to_string(final_img, lang='eng',config='--psm 10 --oem 3 -c tessedit_char_whitelist=BCDFGHIJKLMNPRSTVWZYZ0123456789')
print("Number plate is:", plate)